In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint
import csv
# from api_key import api_key
#from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from pandas import datetime
import matplotlib.pyplot as plt

C:\Users\Bashira\anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  del sys.path[0]


In [2]:
export_url = (f"https://api.census.gov/data/timeseries/intltrade/exports")
import_url = (f"https://api.census.gov/data/timeseries/intltrade/imports")
china_code = "5700"
usa_code = "1000"
month = []
year = []


In [3]:
export_by_commodity = requests.get(f"{export_url}/hs?get=E_COMMODITY,E_COMMODITY_SDESC,ALL_VAL_MO,ALL_VAL_YR&time=from+2015-01&COMM_LVL=HS2&CTY_CODE={china_code}").json()


In [4]:
len(export_by_commodity)

6681

In [5]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 6680:
    count += 1
    movalue.append(export_by_commodity[count][2])
    yrvalue.append(export_by_commodity[count][3])
    desc.append(export_by_commodity[count][1])
    period.append(export_by_commodity[count][4])
    code.append(export_by_commodity[count][0])
china_data_export = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value" : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [6]:
china_data_export.head()

,YTD Value,Monthly Value,Period,Commodity,Comm Code
0,113952886,79435553,2015-02,"FISH, CRUSTACEANS & AQUATIC INVERTEBRATES",03
1,231419675,117466789,2015-03,"FISH, CRUSTACEANS & AQUATIC INVERTEBRATES",03
2,302996552,71576877,2015-04,"FISH, CRUSTACEANS & AQUATIC INVERTEBRATES",03
3,389330885,86334333,2015-05,"FISH, CRUSTACEANS & AQUATIC INVERTEBRATES",03
4,432744545,43413660,2015-06,"FISH, CRUSTACEANS & AQUATIC INVERTEBRATES",03


In [9]:
import_by_commodity = requests.get(f"{import_url}/hs?get=I_COMMODITY,I_COMMODITY_SDESC,GEN_VAL_MO,GEN_VAL_YR&time=from+2015-01&COMM_LVL=HS2&CTY_CODE={china_code}").json()


In [10]:
len(import_by_commodity)

6763

In [11]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 6762:
    count += 1
    movalue.append(import_by_commodity[count][2])
    yrvalue.append(import_by_commodity[count][3])
    desc.append(import_by_commodity[count][1])
    period.append(import_by_commodity[count][4])
    code.append(import_by_commodity[count][0])
china_data_import = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value" : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [12]:
china_data_export["YTD Value"] =pd.to_numeric(china_data_export["YTD Value"])
china_data_import["YTD Value"] =pd.to_numeric(china_data_import["YTD Value"])
china_data_import["Monthly Value"] =pd.to_numeric(china_data_import["Monthly Value"])
china_data_export["Monthly Value"] =pd.to_numeric(china_data_export["Monthly Value"])

In [13]:
def yearly_data(trade,year):
    data = trade[trade["Period"].str.contains(year)]
    data = data.groupby(["Comm Code","Commodity"])["YTD Value"].max()
    data = pd.DataFrame({"total" : data})
#     data = data.nlargest(10,"total")
    data = data.reset_index()
    
    return data
    
    

In [10]:
# test= pd.DataFrame({"total" : data_2015})
china_data_export["Commodity"].nunique()

97

In [14]:
data_2015_import = yearly_data(china_data_import,"2015")
data_2015_export = yearly_data(china_data_export,"2015")

data_2016_import = yearly_data(china_data_import,"2016")
data_2016_export = yearly_data(china_data_export,"2016")

data_2017_import = yearly_data(china_data_import,"2017")
data_2017_export = yearly_data(china_data_export,"2017")

data_2018_import = yearly_data(china_data_import,"2018")
data_2018_export = yearly_data(china_data_export,"2018")

data_2019_import = yearly_data(china_data_import,"2019")
data_2019_export = yearly_data(china_data_export,"2019")

data_2020_import = yearly_data(china_data_import,"2020")
data_2020_export = yearly_data(china_data_export,"2020")

In [20]:
data_2020_export

,Comm Code,Commodity,total
0,01,LIVE ANIMALS,9921930
1,02,MEAT AND EDIBLE MEAT OFFAL,2249435883
2,03,"FISH, CRUSTACEANS & AQUATIC INVERTEBRATES",515820386
3,04,DAIRY PRODS; BIRDS EGGS; HONEY; ED ANIMAL PR N...,231691877
4,05,"PRODUCTS OF ANIMAL ORIGIN, NESOI",201084374
...,...,...,...
92,94,FURNITURE; BEDDING ETC; LAMPS NESOI ETC,125774450
93,95,"TOYS, GAMES & SPORT EQUIPMENT; PARTS & ACCESSO...",88432016
94,96,MISCELLANEOUS MANUFACTURED ARTICLES,49450028
95,97,"WORKS OF ART, COLLECTORS' PIECES AND ANTIQUES",61403204


In [25]:
data_2015_export["total"].sum()

115873365314

In [29]:
change_15_to_16 = round((data_2016_export["total"]-data_2015_export["total"])/data_2015_export["total"],2)
change_16_to_17 = round((data_2017_export["total"]-data_2016_export["total"])/data_2016_export["total"],2)
change_17_to_18 = round((data_2018_export["total"]-data_2017_export["total"])/data_2017_export["total"],2)
change_18_to_19 = round((data_2019_export["total"]-data_2018_export["total"])/data_2018_export["total"],2)
change_19_to_20 = round((data_2020_export["total"]-data_2019_export["total"])/data_2019_export["total"],2)
# pd.set_option('display.long_format', lamba d: '')
df = pd.DataFrame({"code" : data_2015_export["Comm Code"],
                           "commodity" :data_2015_export["Commodity"] ,
                           "total 2015" : data_2015_export["total"],
                           "Change 15 to 16" : change_15_to_16,
                           "total 2016" : data_2016_export["total"],
                           "Change 16 to 17": change_16_to_17,
                           "total 2017": data_2017_export["total"],
                            "Change 17 to 18": change_17_to_18,
                            "total 2018": data_2018_export["total"],
                           "Change 18 to 19": change_18_to_19,
                            "total 19": data_2019_export["total"],
                            "Change 19 to 20": change_19_to_20,
                            "total 2020": data_2020_export["total"]
                          })
df
# test_df_export = df[df["total 2018"] > 1000000000]

,code,commodity,total 2015,Change 15 to 16,total 2016,Change 16 to 17,total 2017,Change 17 to 18,total 2018,Change 18 to 19,total 19,Change 19 to 20,total 2020
0,01,LIVE ANIMALS,10244385,-0.38,6399523,1.61,16726515,-0.16,14028840,-0.62,5277252,0.88,9921930
1,02,MEAT AND EDIBLE MEAT OFFAL,335616368,0.74,584610550,-0.11,521899634,-0.25,391174508,2.03,1186680064,0.90,2249435883
2,03,"FISH, CRUSTACEANS & AQUATIC INVERTEBRATES",1007942885,-0.06,949230937,0.30,1235726422,-0.14,1064292878,-0.19,862740560,-0.40,515820386
3,04,DAIRY PRODS; BIRDS EGGS; HONEY; ED ANIMAL PR N...,293604977,-0.10,265611120,0.54,407723006,-0.23,313032859,-0.40,187779839,0.23,231691877
4,05,"PRODUCTS OF ANIMAL ORIGIN, NESOI",266200493,-0.05,254085015,0.05,267390973,0.19,317097034,-0.07,295931963,-0.32,201084374
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,94,FURNITURE; BEDDING ETC; LAMPS NESOI ETC,314639121,-0.09,286389581,-0.02,281043110,0.04,291045131,-0.22,227023585,-0.45,125774450
93,95,"TOYS, GAMES & SPORT EQUIPMENT; PARTS & ACCESSO...",160206404,0.14,182011147,0.05,190405597,0.03,195538230,0.02,200131126,-0.56,88432016
94,96,MISCELLANEOUS MANUFACTURED ARTICLES,54778354,0.49,81392996,-0.15,68835453,0.05,72273621,-0.14,62262773,-0.21,49450028
95,97,"WORKS OF ART, COLLECTORS' PIECES AND ANTIQUES",101272167,-0.17,83671628,0.11,92770907,0.18,109557547,0.73,189283364,-0.68,61403204


In [30]:
df.to_csv("change_over_year.csv", index=False)

In [19]:
# test_df_export.round(2)


In [21]:
# data_2015= test.nlargest(10,"total")
# data_2015_import = data_2015_import.to_csv("data_2015_import.csv", index=False)
# data_2015_export = data_2015_export.to_csv("data_2015_export.csv", index=False)

# data_2016_import = data_2016_import.to_csv("data_2016_import.csv", index=False)
# data_2016_export = data_2016_export.to_csv("data_2016_export.csv", index=False)

# data_2017_import = data_2017_import.to_csv("data_2017_import.csv", index=False)
# data_2017_export = data_2017_export.to_csv("data_2017_export.csv", index=False)

# data_2018_import = data_2018_import.to_csv("data_2018_import.csv", index=False)
# data_2018_export = data_2018_export.to_csv("data_2018_export.csv", index=False)

In [53]:
total_import_export =  pd.DataFrame({"year":["2015","2016","2017","2018","2019","2020"],
                                     "import":[data_2015_import["total"].sum(),data_2016_import["total"].sum(),data_2017_import["total"].sum(),data_2018_import["total"].sum(),data_2019_import["total"].sum(),data_2020_import["total"].sum()],
                                     "export":[data_2015_export["total"].sum(),data_2016_export["total"].sum(),data_2017_export["total"].sum(),data_2018_export["total"].sum(),data_2019_export["total"].sum(),data_2020_export["total"].sum()]
                                     
})
total_import_export

,year,import,export
0,2015,483201655364,115873365314
1,2016,462419991603,115594784477
2,2017,505165098057,129997227243
3,2018,539243139431,120289287877
4,2019,451651438113,106447251090
5,2020,303889567057,81100662652


In [22]:
total_import_export["deficit"]= total_import_export["import"]-total_import_export["export"]

In [54]:
total_import_export.to_csv("new_import_export.csv", index=False)

In [23]:
# data_2015 = data_2015.to_csv("data_2015.csv", index=False)

In [58]:
monthly_export = pd.DataFrame({"month":["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],
                                     "2015 export":monthly_2015_export["total"],
                                     "2016 export":monthly_2016_export["total"],
                                     "2017 export":monthly_2017_export["total"],
                                     "2018 export":monthly_2018_export["total"],
                                     "2019 export":monthly_2019_export["total"],
                                     "2020 export":monthly_2020_export["total"]
                                      })
monthly_export

,month,2015 export,2016 export,2017 export,2018 export,2019 export,2020 export
0,Jan,9424681061,8208886377,9955522802,9910227741,7105125159,7.215277e+09
1,Feb,8754588178,8080515063,9739814915,9741752846,8083304442,6.815003e+09
2,Mar,9886519164,8925561211,9720184109,12653174318,10574937623,7.971890e+09
3,Apr,9279912850,8679668476,9806476826,10510478379,7883015916,8.604695e+09
4,May,8749787563,8541963035,9880023522,10396624247,9069418815,9.641702e+09
5,Jun,9615754115,8845557796,9718210931,10858294665,9166696315,9.242238e+09
6,Jul,9505173559,9129666281,9954063313,10156527887,8694298164,9.037041e+09
7,Aug,9183456081,9372919258,10825460770,9280892425,9415647335,1.103607e+10
8,Sep,9419421166,9521151936,10895979304,9732404733,8597272439,1.153675e+10
9,Oct,11324856248,12600004710,12963222187,9187533643,8851210888,NaN


In [60]:
monthly_import = pd.DataFrame({"month":["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],
                                     "2015 import":monthly_2015_import["total"],
                                     "2016 import":monthly_2016_import["total"],
                                     "2017 import":monthly_2017_import["total"],
                                     "2018 import":monthly_2018_import["total"],
                                     "2019 import":monthly_2019_import["total"],
                                     "2020 import":monthly_2020_import["total"]
                                      })
monthly_import

,month,2015 import,2016 import,2017 import,2018 import,2019 import,2020 import
0,Jan,38337369698,37126434120,41335553396,45749942367,41514449304,3.328057e+10
1,Feb,31563988155,36066892525,32784957412,39003603360,33154912095,2.281306e+10
2,Mar,41136917530,29812326908,34161993661,38295105288,31175576988,1.980543e+10
3,Apr,36121394467,32920204539,37441870879,38269375930,34682652223,3.107081e+10
4,May,39082029905,37513726589,41756844722,43938652758,39173360876,3.659817e+10
5,Jun,41453839795,38539190214,42258078621,44571234982,38967641814,3.763948e+10
6,Jul,41215078176,39438857738,43561110857,47087587600,41449210496,4.065728e+10
7,Aug,44138055272,43221774205,45782311949,47817451399,41151143674,4.081644e+10
8,Sep,45724951242,42020928547,45405051275,49988069149,40165462896,4.120833e+10
9,Oct,44309932520,43798098424,48133149629,52170086089,40114878589,NaN


In [59]:
monthly_export.to_csv("monthly_export.csv", index=False)

In [61]:
monthly_import.to_csv("monthly_import.csv", index=False)

In [55]:
def monthly_data(trade,year):
    month = trade[trade["Period"].str.contains(year)]
    month = month.groupby(["Period"])["Monthly Value"].sum()
    month = pd.DataFrame({"total" : month})
#     data = data.nlargest(10,"total")
    month = month.reset_index()
    
    return month

In [56]:
monthly_2015_export = monthly_data(china_data_export,"2015")
monthly_2015_import = monthly_data(china_data_import,"2015")

monthly_2016_import = monthly_data(china_data_import,"2016")
monthly_2016_export = monthly_data(china_data_export,"2016")

monthly_2017_import = monthly_data(china_data_import,"2017")
monthly_2017_export = monthly_data(china_data_export,"2017")

monthly_2018_import = monthly_data(china_data_import,"2018")
monthly_2018_export = monthly_data(china_data_export,"2018")

monthly_2019_import = monthly_data(china_data_import,"2019")
monthly_2019_export = monthly_data(china_data_export,"2019")

monthly_2020_import = monthly_data(china_data_import,"2020")
monthly_2020_export = monthly_data(china_data_export,"2020")

In [41]:
Us_total_export = china_data_export.to_csv("US_total_export.csv", index=False)

In [67]:
US_total_import = china_data_import.to_csv("US_total_impor.csv", index=False)

In [ ]:
# Simple Linear regression

In [46]:
tt = china_data_export[china_data_export["Period"].str.contains("2019")]


In [47]:
tt["Monthly Value"].sum()

106447251090